''''sh
 pip install requests ipykernel Pillow     
''''

In [2]:
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime
import textwrap
import os

base_dpi = 300  # Base DPI for standard displays
current_dpi = 300  # Desired DPI for rendering


def generate_image(crop, location,Texts=None):
    today_date = datetime.today().strftime("%Y-%m-%d")
    title = f"Risk Factors - {today_date}\n{crop} - {location}"

    # Create a blank image with higher resolution
    img = Image.new("RGB", (1200, 1600), "#A5D6A7")  # Increased resolution to 1200x1600
    draw = ImageDraw.Draw(img)

    # Scale font sizes based on DPI
    font_title_size = int(56 * (current_dpi / base_dpi))
    font_header_size = int(40 * (current_dpi / base_dpi))
    font_text_size = int(40 * (current_dpi / base_dpi))

    font_title = ImageFont.load_default(font_title_size)
    font_header = ImageFont.load_default(font_header_size)
    font_text = ImageFont.load_default(font_text_size)

    # Draw the title background   
    draw.rounded_rectangle([(0, -20), (1200, 220)], 
                               radius=20, fill="#2E7D32", outline="#1B5E20", width=3)

    # Draw title with padding and center it
    title_lines = title.split("\n")
    title_y = 60
    for i, line in enumerate(title_lines):
        text_width = draw.textlength(line, font=font_title)  # Get the width of the text
        text_x = (1200 - text_width) // 2  # Center the title horizontally
        draw.text((text_x, title_y + i * 80), line, fill="black", font=font_title)

    y_offset = 260
    bubble_width = 1000

    # Increased padding for more space
    padding = 40  # Increased padding inside the bubbles
    bubble_padding = 40  # Increased space between each bubble
    text_padding = 40  # Padding for text inside the bubble

    elements=[
    {"type":"Temp","header": "Elevated temperature", "body": "The maximum temperature will be exceeded, I recommend applyig Stress Buster to ensure the well being of your crops.", "type": "warning", "icon": "Icons/temperature-arrow-up-solid.png"},
    {"type":"Moist","header": "Low moisture", "body": "A drought is incoming, I recomend appling Stress buster to ensure the well being of your crops.", "type": "warning", "icon": "Icons/sun-plant-wilt-solid.png"},
    {"type":"Frost","header": "Frost Warning", "body": "Frost is expected in the coming days, I recommend applying Stress Buster to your crops to ensure their well being", "type": "warning", "icon": "Icons/snowflake-regular.png"},
    {"type":"Yield","header": "Yield Risk", "body":"Based on avilible data, your yield is at a risk of being lower that expected. Apply Yield Booster to your crops to ensure the best possible yield.", "type": "warning", "icon": "Icons/arrow-down-wide-short-solid.png"}
    ]

    for i in range(len(elements)):

        header, body, element_type, icon_path = elements[i]["header"], elements[i]["body"], elements[i]["type"], elements[i]["icon"]

        # Fix file path issue (Windows vs Linux)
        icon_path = icon_path.replace("\\", "/")

        # Load the icon from a local file
        if os.path.exists(icon_path):
            icon = Image.open(icon_path).convert("RGBA").resize((80, 80))  # Resize icon for higher resolution
        else:
            print(f"⚠️ Warning: Icon '{icon_path}' not found!")  # Debug message
            icon = None  # Prevent crash

        # Set color based on type
        box_color = "#E53935" if element_type == "warning" else "#FDD835" if element_type == "info" else "#43A047"

        # Wrap body text with an adjusted width so it fits within the bubble
        wrapped_body = textwrap.fill(body, width=45)

        # Calculate text sizes correctly
        header_bbox = draw.textbbox((0, 0), header, font=font_header)  # Get bounding box
        header_height = header_bbox[3] - header_bbox[1]

        body_lines = wrapped_body.split("\n")
        body_height = sum([draw.textbbox((0, 0), line, font=font_text)[3] - draw.textbbox((0, 0), line, font=font_text)[1] for line in body_lines])

        # Adjust bubble height dynamically with padding
        bubble_height = header_height + body_height + 2 * padding

        # Calculate x position for centering the bubble
        bubble_x = (1200 - bubble_width) // 2

        # Shadow offset and color
        shadow_offset = 10
        shadow_color = (0, 0, 0, 150)  # Semi-transparent black for the shadow

        # Draw shadowed bubble (offset slightly)
        draw.rounded_rectangle([(bubble_x + shadow_offset, y_offset + shadow_offset), 
                                (bubble_x + bubble_width + shadow_offset, y_offset + bubble_height + shadow_offset)], 
                            radius=20, fill=shadow_color)

        # Draw the actual bubble on top
        draw.rounded_rectangle([(bubble_x, y_offset), (bubble_x + bubble_width, y_offset + bubble_height)], 
                            radius=20, fill=box_color, outline="black", width=3)

        text_x = bubble_x + text_padding  # Adjust text start position after icon

        # Paste icon (if available)
        if icon:
            img.paste(icon, (text_x, y_offset + 20), icon)  # Paste the icon on the image with transparency
            text_x += 100  # Adjust text to avoid overlapping icon

        # Draw header text with padding
        draw.text((text_x, y_offset + 20), header, fill="black", font=font_header)

        # Draw body text with increased line spacing and padding
        line_height = draw.textbbox((0, 0), "A", font=font_text)[3] - draw.textbbox((0, 0), "A", font=font_text)[1]
        for i, line in enumerate(body_lines):
            line_y = y_offset + header_height + padding + i * (line_height + 10)  # Added 10 pixels for extra line spacing
            draw.text((text_x, line_y), line, fill="black", font=font_text)

        # Update y_offset to position next bubble with additional padding
        y_offset += bubble_height + bubble_padding
    img.save("output_image.png", dpi=(current_dpi, current_dpi))
    img.show()

# Example input data
crop = "Corn"
location = "St. Gallen, Switzerland"

# generate elements based on recomendations:


#risks={'diurnal_stress': 0, 'nighttime_stress': 0, 'frost_stress': 0, 'drought_risk': 1, 'yield_risk': 1}
generate_image(crop, location)